In [ ]:
%pip install librosa

In [7]:
import pandas as pd
import librosa
import numpy as np
import os

In [9]:
genres = [
    "gaming",
    "true_crime",
    "business",
    "science",
    "self_help",
    "comedy",
    "sports",
]

n_samples = 51
data = np.zeros((35, 2 + 2*n_samples), dtype=object)
data_idx = 0

for genre in genres:
    file_names = os.listdir("../data/" + genre + "/")
    print(f"started {genre}")

    for file_name in file_names:
        y, sr = librosa.load("../data/" + genre + "/" + file_name)
        pitches, _ = librosa.piptrack(y=y, sr=sr)

        data[data_idx, 0] = genre
        data[data_idx, 1] = file_name

        nonzero_mask = pitches != 0
        # average the pitches over time
        column_averages = np.nanmean(
            np.where(nonzero_mask, pitches, np.nan), axis=0
        )
        pitches_processed = np.nan_to_num(column_averages, nan=0)

        # grab the pitches at 0, 5, 10, ..., 100 percent
        final_pitches = []
        n = len(pitches_processed) - 1
        for idx in np.linspace(0, len(pitches_processed)-1, num=n_samples, endpoint=True, dtype=int):
            # data is in the form [pitch, percent] where percent is the time in the audio
            final_pitches.append(pitches_processed[idx])

        # file.write(genre + ", " + file_name + ", " + str(final_pitches) + "\n")

        final_pitches = np.array(final_pitches)
        data[data_idx, 2:2+n_samples] = final_pitches

        volume = librosa.feature.rms(y=y)

        # grab the volumes at 0, 5, 10, ..., 100 percent
        final_volumes = []
        n = volume.shape[1] - 1
        for idx in np.linspace(0, n, num=n_samples, endpoint=True, dtype=int):
            # data is in the form [volume, percent] where percent is the time in the audio
            final_volumes.append(np.sqrt(volume[0][idx]))

        # file.write(genre + ", " + file_name + ", " + str(final_volumes) + "\n")
        final_volumes = np.array(final_volumes)
        data[data_idx, 2+n_samples:] = final_volumes

        data_idx += 1
        # totals += final_volumes[:]

started gaming


/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_80643/2235220494.py:28: RuntimeWarning: Mean of empty slice
  column_averages = np.nanmean(


started true_crime
started business
started science
started self_help
started comedy
started sports


In [ ]:
def calc_volume(file):
    genres = [
        "gaming",
        "true_crime",
        "business",
        "science",
        "self_help",
        "comedy",
        "sports",
    ]

    for genre in genres:
        file_names = os.listdir("data/" + genre + "/")
        totals = np.zeros(21)

        for file_name in file_names:
            y, _ = librosa.load("data/" + genre + "/" + file_name)
            volume = librosa.feature.rms(y=y)

            # grab the volumes at 0, 5, 10, ..., 100 percent
            final_volumes = []
            n = volume.shape[1] - 1
            for percent in np.linspace():
                ind = int(np.floor(n * (percent / 100)))
                # data is in the form [volume, percent] where percent is the time in the audio
                final_volumes.append(np.sqrt(volume[0][ind]))

            file.write(genre + ", " + file_name + ", " + str(final_volumes) + "\n")

            final_volumes = np.array(final_volumes)
            totals += final_volumes[:]

        # calculate the average volume for the current genre
        totals = totals / 5
        i = 0
        averages_percents = []
        for percent in range(0, 105, 5):
            averages_percents.append(totals[i])
            i += 1

    return

In [10]:
columns = ['genre', 'name']
for metric in ['pitch', 'volume']:
  for p in range(n_samples):
    columns.append(f"{metric} - {p}")

# assert len(columns) == 44

In [11]:
pitch_cols = [f"pitch - {p}" for p in range(n_samples)]
vol_cols = [f"volume - {p}" for p in range(n_samples)]
print(pitch_cols)

['pitch - 0', 'pitch - 1', 'pitch - 2', 'pitch - 3', 'pitch - 4', 'pitch - 5', 'pitch - 6', 'pitch - 7', 'pitch - 8', 'pitch - 9', 'pitch - 10', 'pitch - 11', 'pitch - 12', 'pitch - 13', 'pitch - 14', 'pitch - 15', 'pitch - 16', 'pitch - 17', 'pitch - 18', 'pitch - 19', 'pitch - 20', 'pitch - 21', 'pitch - 22', 'pitch - 23', 'pitch - 24', 'pitch - 25', 'pitch - 26', 'pitch - 27', 'pitch - 28', 'pitch - 29', 'pitch - 30', 'pitch - 31', 'pitch - 32', 'pitch - 33', 'pitch - 34', 'pitch - 35', 'pitch - 36', 'pitch - 37', 'pitch - 38', 'pitch - 39', 'pitch - 40', 'pitch - 41', 'pitch - 42', 'pitch - 43', 'pitch - 44', 'pitch - 45', 'pitch - 46', 'pitch - 47', 'pitch - 48', 'pitch - 49', 'pitch - 50']


In [12]:
import json
df = pd.DataFrame(data, columns=columns)
pitch_data = df.drop(columns=vol_cols)
vol_data = df.drop(columns=pitch_cols)
genre_pitch = {}
genre_volume = {}
for genre in genres:
    genre_pitch[genre] = (
        pitch_data[pitch_data["genre"] == genre].mean(numeric_only=None).to_list()
    )
    genre_volume[genre] = (
        vol_data[vol_data["genre"] == genre].mean(numeric_only=None).to_list()
    )

/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_80643/2022912262.py:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pitch_data[pitch_data["genre"] == genre].mean(numeric_only=None).to_list()
/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_80643/2022912262.py:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  vol_data[vol_data["genre"] == genre].mean(numeric_only=None).to_list()
/var/folders/pw/_mnnd_wx7_qb32qhrnpgm8ww0000gn/T/ipykernel_80643/2022912262.py:9: FutureWarning: The default value of numeric_only in

In [ ]:
pitch_data.mean()

In [ ]:
vol_data.mean()

In [ ]:
all_data_tree = {}
for genre in genres:
    this_genre = {}
    for idx, row in df[df['genre'] == genre].iterrows():
        this_genre[row['name']] = {}
        this_genre[row["name"]]['pitch'] = row[pitch_cols].to_list()
        this_genre[row["name"]]['volume'] = row[vol_cols].to_list()
    all_data_tree[genre] = this_genre

In [ ]:
with open("./processed_data/aggregate.json", "w") as file:
  json.dump(all_data_tree, file)

In [ ]:
averages = {"pitch": genre_pitch, "volume": genre_volume}
with open("./processed_data/averages.json", "w") as file:
    json.dump(averages, file)

In [ ]:
df[pitch_cols].max(axis=None).min()

In [ ]:
y, sr = librosa.load("data/gaming/The Beginning Begins (Ep. 1) ｜ Fantasy High.mp3")
pitches, _ = librosa.piptrack(y=y, sr=sr)


In [ ]:
nonzero_mask = pitches != 0
        # average the pitches over time
column_averages = np.nanmean(
    np.where(nonzero_mask, pitches, np.nan), axis=0
)
pitches_processed = np.nan_to_num(column_averages, nan=0)

print(pitches_processed.shape)

In [ ]:
all_data_tree

In [31]:
pretty = {
    "gaming": "Gaming",
    "true_crime": "True Crime",
    "business": "Business",
    "science": "Science",
    "self_help": "Self Help",
    "comedy": "Comedy",
    "sports": "Sports",
}

# avg_data_rep_2 = {"pitch": [], "volume": []}
# for i in range(n_samples):
#     pitches = {
#         "idx": np.round(i / (n_samples-1) * 100),
#     }
#     volumes = {
#         "idx": np.round(i / (n_samples-1) * 100),
#     }
#     volumes.update({f"{pretty[genre]}": genre_volume[genre][i] for genre in genres})
#     pitches.update({f"{pretty[genre]}": genre_pitch[genre][i] for genre in genres})
#     avg_data_rep_2["pitch"].append(pitches)
#     avg_data_rep_2["volume"].append(volumes)

# print(avg_data_rep_2["volume"])

In [ ]:
avg_data_rep_2[5]

In [ ]:
genre_pitch = {}
genre_volume = {}
for genre in genres:
    genre_pitch[genre] = (
        pitch_data[pitch_data["genre"] == genre].mean(numeric_only=None).to_list()
    )
    genre_volume[genre] = (
        vol_data[vol_data["genre"] == genre].mean(numeric_only=None).to_list()
    )

In [ ]:
all_data = {}
for genre in genres:
    relevant = df[df['genre'] == genre]
    genre_data = {"pitch": [], "volume": []}
    for i in range(n_samples):
        pitches = {
          "idx": np.round(i / (n_samples - 1) * 100),
      }
        volumes = {
          "idx": np.round(i / (n_samples - 1) * 100),
      }
        volumes.update(
            {ep["name"][0:-4]: ep[f"volume - {i}"] for _, ep in relevant.iterrows()}
        )
        pitches.update(
            {ep["name"][0:-4]: ep[f"pitch - {i}"] for _, ep in relevant.iterrows()}
        )
        genre_data["pitch"].append(pitches)
        genre_data["volume"].append(volumes)
    all_data[genre] = genre_data

print(all_data)

In [ ]:
with open("./src/data/all_data_lists.json", "w") as f:
    json.dump(all_data, f)

In [ ]:
with open("./src/data/averages_list.json", "w") as f:
  json.dump(avg_data_rep_2, f) 

In [ ]:
name_map = {}
for genre in genres:
    name_map[genre] = [name[0:-4] for name in df[df["genre"] == genre]['name'].to_list()]

with open("./src/data/name_lists.json", "w") as f:
    json.dump(name_map, f)

In [ ]:
all_data

In [32]:
average_pitch = df.groupby('genre')[pitch_cols].mean().mean(axis=1)
print(average_pitch)

avg_vol = df.groupby("genre")[vol_cols].mean().mean(axis=1)
print(avg_vol.to_dict())

averages = []
for genre in genres:
  averages.append({
    "genre": pretty[genre],
    "pitch": average_pitch.to_dict()[genre],
    "volume": avg_vol.to_dict()[genre]
  })

print(averages)

with open("../src/data/avg_over_time.json", "w") as f:
  json.dump(averages,f)

genre
business       737.726053
comedy        1085.500291
gaming        1127.886959
science       1027.431276
self_help      836.173850
sports        1018.376277
true_crime     844.301060
dtype: float64
{'business': 0.22671536625010258, 'comedy': 0.24662683042116854, 'gaming': 0.21597508707259183, 'science': 0.2122420282596174, 'self_help': 0.23693260459923268, 'sports': 0.2532266702525792, 'true_crime': 0.2271666572176906}
[{'genre': 'Gaming', 'pitch': 1127.886959479837, 'volume': 0.21597508707259183}, {'genre': 'True Crime', 'pitch': 844.3010600968904, 'volume': 0.2271666572176906}, {'genre': 'Business', 'pitch': 737.7260532154756, 'volume': 0.22671536625010258}, {'genre': 'Science', 'pitch': 1027.4312762690524, 'volume': 0.2122420282596174}, {'genre': 'Self Help', 'pitch': 836.1738501455269, 'volume': 0.23693260459923268}, {'genre': 'Comedy', 'pitch': 1085.5002911137603, 'volume': 0.24662683042116854}, {'genre': 'Sports', 'pitch': 1018.3762768913718, 'volume': 0.2532266702525792}]
